In [1]:
#R. Bickley, M. Kramer, J. Jacobs, 2019 for Radio Frequency Systems
#Accepts unprocessed profile data
#Calculates sizes and depths
#Returns measurements

In [11]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

In [2]:
def read_in(filename):
    #file should be a .csv with columns: 'ind','p1','p2','p3'...'pn' where n is the number of tuners measured
    dat = pd.read_csv(filename)
    return dat

In [6]:
def plane_eqn(points):
    #where points is an array with dimensions:
    #[[xa,ya,za],[xb,yb,zb],[xc,yc,zc]]
    #check array dimensions:
    if points.shape != (3,3):
        return('Points array does not have the required dimensions.')
    else:
        p_a = points[0]
        p_b = points[1]
        p_c = points[2]
        #construct the vectors
        v_ab = np.array([[p_b[0]-p_a[0]],[p_b[1]-p_a[1]],[p_b[2]-p_a[2]]])
        v_ac = np.array([[p_c[0]-p_a[0]],[p_c[1]-p_a[1]],[p_c[2]-p_a[2]]])
        plane = np.cross(v_ab,v_ac,axis = 0)
        if np.count_nonzero(plane) == 0:
            return('Points are collinear.')
        else:
            d = -plane[0]*p_a[0]-plane[1]*p_a[1]-plane[2]*p_a[2]
            plane = np.array([plane[0],plane[1],plane[2],d])
            return plane

In [7]:
def local_pcb_height(plane,point):
    #where plane is the 1x4 vector and point is a x, y coordinate pair
    if plane.shape != (4,1):
        return('Plane vector does not have the required dimensions')
    if len(point) != (2):
        return('Point is not a coordinate pair')
    else:
        z_loc = (-plane[0]*point[0]-plane[1]*point[1]-plane[3])/plane[2]
        return(z_loc)

In [4]:
def calculate(dat):
    #gets the size and depth for each profile column in the given data
    tuner_sizes = []
    tuner_depths = []
    for i in range(len(dat)-6):
        #Performing a polynomial fit on the (i+1)th tuner profile
        x_new = np.linspace(dat['ind'][0], dat['ind'][100], num=len(dat['ind'])*10)
        coefs = np.polyfit(dat['ind'], dat['p'+str(i+1)], deg=15)
        ffit = np.polyval(coefs, x_new)

        #Get and sort the local maxima of the fit
        loc_max = ffit[argrelextrema(ffit, np.greater)[0]]
        loc_max.sort()
        #Get and sort the local minima of the fit
        loc_min = ffit[argrelextrema(ffit, np.less)[0]]
        loc_min.sort()

        fids = [dat['fid1'],dat['fid2'],dat['fid3']]
        point = [dat['x_coords'][i],dat['y_coords'][i]]
        local_zero = local_pcb_height(plane_eqn(fids),point)
        #Identify the tuner bottom as the position of the largest maximum
        tuner_bottom = loc_max[-1]+0.012645833 # adding the tuner bottom thickness to the largest maximum
        #Identify the tuner top as the average of the positions of the two largest minima, either side of the tuner well
        tuner_top = np.average(loc_min[0:2])
        tuner_size = (tuner_bottom - tuner_top)
        tuner_depth = tuner_bottom - local_zero

        tuner_sizes.append(tuner_size)
        tuner_depths.append(tuner_depth)

    return tuner_sizes, tuner_depths

In [10]:
def make_sheet(dat):
    #This outputs the finished excel file
    sizes, depths = calculate(dat)
    indices = []
    for i in range(len(dat)-6):
        indices.append('p'+str(i+1))
    df_out = pd.DataFrame({'Index':indices,'Size':sizes,'Depth':depths})
    df_out.to_excel('tuner_measurements.xlsx', sheet_name='sheet1', index=False)

In [ ]:
#Input the name of the formatted measurement file below
make_sheet(read_in(filename))